In [1]:
import csv

import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split

RANDOM_SEED = 42

2023-04-27 16:42:19.374007: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-27 16:42:19.718422: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-27 16:42:19.719902: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-27 16:42:20.784663: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


# Specify each path

In [2]:
dataset = 'model/keypoint_classifier/keypoint.csv'
model_save_path = 'model/keypoint_classifier/keypoint_classifier.hdf5'
tflite_save_path = 'model/keypoint_classifier/keypoint_classifier.tflite'

# Set number of classes

In [3]:
NUM_CLASSES = 9

# Dataset reading

In [4]:
X_dataset = np.loadtxt(dataset, delimiter=',', dtype='float32', usecols=list(range(1, (21 * 2) + 1)))

In [5]:
y_dataset = np.loadtxt(dataset, delimiter=',', dtype='int32', usecols=(0))

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X_dataset, y_dataset, train_size=0.75, random_state=RANDOM_SEED)

# Model building

In [7]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Input((21 * 2, )),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(80, activation='relu'),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(NUM_CLASSES, activation='softmax')
])

In [8]:
model.summary()  # tf.keras.utils.plot_model(model, show_shapes=True)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dropout (Dropout)           (None, 42)                0         
                                                                 
 dense (Dense)               (None, 80)                3440      
                                                                 
 dropout_1 (Dropout)         (None, 80)                0         
                                                                 
 dense_1 (Dense)             (None, 10)                810       
                                                                 
 dense_2 (Dense)             (None, 9)                 99        
                                                                 
Total params: 4,349
Trainable params: 4,349
Non-trainable params: 0
_________________________________________________________________


In [9]:
# Model checkpoint callback
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    model_save_path, verbose=1, save_weights_only=False)
# Callback for early stopping
es_callback = tf.keras.callbacks.EarlyStopping(patience=20, verbose=1)

In [10]:
# Model compilation
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Model training

In [11]:
model.fit(
    X_train,
    y_train,
    epochs=1000,
    batch_size=128,
    validation_data=(X_test, y_test),
    callbacks=[cp_callback, es_callback]
)

Epoch 1/1000
 1/41 [..............................] - ETA: 24s - loss: 2.2779 - accuracy: 0.1484
Epoch 1: saving model to model/keypoint_classifier/keypoint_classifier.hdf5
41/41 [==============================] - 1s 6ms/step - loss: 2.0986 - accuracy: 0.1988 - val_loss: 1.9193 - val_accuracy: 0.2915
Epoch 2/1000
40/41 [============================>.] - ETA: 0s - loss: 1.8129 - accuracy: 0.3225
Epoch 2: saving model to model/keypoint_classifier/keypoint_classifier.hdf5
41/41 [==============================] - 0s 2ms/step - loss: 1.8103 - accuracy: 0.3243 - val_loss: 1.5945 - val_accuracy: 0.4303
Epoch 3/1000
 1/41 [..............................] - ETA: 0s - loss: 1.6308 - accuracy: 0.3906
Epoch 3: saving model to model/keypoint_classifier/keypoint_classifier.hdf5
41/41 [==============================] - 0s 2ms/step - loss: 1.5202 - accuracy: 0.4279 - val_loss: 1.3410 - val_accuracy: 0.5180
Epoch 4/1000
41/41 [==============================] - ETA: 0s - loss: 1.3090 - accuracy: 0.4858


In [12]:
# Model evaluation
val_loss, val_acc = model.evaluate(X_test, y_test, batch_size=128)

14/14 [==============================] - 0s 1ms/step - loss: 0.1528 - accuracy: 0.9797


In [13]:
# Loading the saved model
model = tf.keras.models.load_model(model_save_path)

In [14]:
# Inference test
predict_result = model.predict(np.array([X_test[0]]))
print(np.squeeze(predict_result))
print(np.argmax(np.squeeze(predict_result)))

1/1 [==============================] - 0s 58ms/step
[5.8803074e-10 2.5982598e-07 2.1694658e-11 9.7260362e-01 2.7208427e-02
 1.8176727e-04 5.5406790e-06 1.7964950e-09 3.3840027e-07]
3


# Saving Model

In [15]:
# Save as a model dedicated to inference
model.save(model_save_path, include_optimizer=False)

In [16]:
# Transform model (quantization)

converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_quantized_model = converter.convert()

open(tflite_save_path, 'wb').write(tflite_quantized_model)

2023-04-27 16:42:44.928291: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'input_1' with dtype float and shape [?,42]
	 [[{{node input_1}}]]
2023-04-27 16:42:44.981028: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,42]
	 [[{{node inputs}}]]
2023-04-27 16:42:44.993006: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'input_1' with dtype float and shape [?,42]
	 [[{{node input_1}}]]
2023-04-27 16:4

INFO:tensorflow:Assets written to: /tmp/tmp82dyk4k9/assets


INFO:tensorflow:Assets written to: /tmp/tmp82dyk4k9/assets
2023-04-27 16:42:45.942360: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:364] Ignored output_format.
2023-04-27 16:42:45.942393: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:367] Ignored drop_control_dependency.
2023-04-27 16:42:45.943163: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /tmp/tmp82dyk4k9
2023-04-27 16:42:45.944240: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2023-04-27 16:42:45.944256: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info (if present) from: /tmp/tmp82dyk4k9
2023-04-27 16:42:45.947687: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:353] MLIR V1 optimization pass is not enabled
2023-04-27 16:42:45.948676: I tensorflow/cc/saved_model/loader.cc:231] Restoring SavedModel bundle.
2023-04-27 16:42:45.988370: I tensorflow/cc/saved_model/loader.cc:215] Running initializatio

9504

# Inference test

In [17]:
interpreter = tf.lite.Interpreter(model_path=tflite_save_path)
interpreter.allocate_tensors()

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [18]:
# Get I / O tensor
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

In [19]:
interpreter.set_tensor(input_details[0]['index'], np.array([X_test[0]]))

In [20]:
%%time
# Inference implementation
interpreter.invoke()
tflite_results = interpreter.get_tensor(output_details[0]['index'])

CPU times: user 1.18 ms, sys: 77 µs, total: 1.26 ms
Wall time: 2.9 ms


In [21]:
print(np.squeeze(tflite_results))
print(np.argmax(np.squeeze(tflite_results)))

[5.1678084e-10 2.2847766e-07 1.8742707e-11 9.7236419e-01 2.7455818e-02
 1.7441719e-04 4.9910755e-06 1.6621927e-09 3.1184823e-07]
3
